In [1]:
#data science and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

#creation of dataset
import _lib.ml_workflow.create_dataset as cds
from _lib.export import to_csv
from _lib.raman_lib.misc import load_data

#quality control
import _lib.ml_workflow.quality_control as qc
from _lib.raman_lib.preprocessing import RangeLimiter
from _lib.raman_lib.visualization import plot_spectra_peaks
from _lib.raman_lib.spectra_scoring import score_names

#preprocessing
from _lib.ml_workflow.preprocess_data import preprocess

#model creation
from sklearn.model_selection import LeaveOneGroupOut, cross_validate, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.cluster import FeatureAgglomeration
from sklearn.decomposition import PCA, NMF
from sklearn.pipeline import Pipeline
from _lib.raman_lib.preprocessing import PeakPicker

#file handling
from pathlib import Path
import os

In [2]:
# define the paths to all experiment data
# each dir contains all the files associated with that experiment
# each file has a prefix that indicates the group and the measurement --> group_measurement_.....
path_etoposide = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide"
path_resveratrol = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Resveratrol"
path_both = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Both"
path_control = "/Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Control"

# Define parameters
In order to function properly, the provided code depends on predefined parameters like output paths, limits and thresholds for the quality control, ...
## Define data paths
Define the location of the data, and where quality-controlled and preprocessed data should be stored. Both of them rely on a unique file-prefix that describes the data being analyzed.

In [3]:
FILE_PREFIX = "HL_428_C_E"
DATASET_OUT = "./data/" + FILE_PREFIX + ".csv"
RESULT_DIR = "./result/" + FILE_PREFIX
QC_OUT = RESULT_DIR + "/" + FILE_PREFIX + "_qc.csv"
PREP_OUT = RESULT_DIR + "/" + FILE_PREFIX + "_preprocessed.csv"
LDA_DIR = RESULT_DIR + "/" + "lda_dim_reduction"
REG_DIR = RESULT_DIR + "/" + "regularized_models/"
TREE_DIR = RESULT_DIR + "/" + "tree_based_models/"

## Define quality scoring parameters
The quality control only uses peaks in a given interval, recognizes peaks via a filter (Sav-Gol) and scores them based on some metrics. Finally, the best N spectra are selected.
### Spectral Range Limits

In [4]:
QC_LIM_LOW = 450
QC_LIM_HIGH = 1650

### Peak Detection

In [5]:
QC_WINDOW = 35
QC_THRESHOLD = 0.001
QC_MIN_HEIGHT = 50

### Scoring

In [6]:
QC_SCORE = 1
QC_PEAKS = 1

### Number of spectra to keep

In [7]:
QC_NUM = 300

## Define Preprocessing Parameter
### Spectral Range Limits

In [8]:
PREP_LIM_LOW = QC_LIM_LOW
PREP_LIM_HIGH = QC_LIM_HIGH

### Window-width for smoothing

In [9]:
PREP_WINDOW = 15

## Settings for Cross Validation

In [10]:
SCORING = ['roc_auc', 'accuracy', 'f1']
N_TRIALS = 20
N_FOLDS = 5
N_CORES = -1

# Create the dataset
Create the dataset using the implementation provided by D. Zimmermann.
For the creation of the dataset, the two source dirs, as well as the desired labels are needed.
Furthermore, an output directory is needed, to store the created dataset 

In [11]:
datadir = Path(DATASET_OUT).parent
if not os.path.exists(datadir):
    os.makedirs(datadir)

dataset = cds.create_dataset([path_control, path_etoposide], ['Control', 'Etoposide'], grouped=True)
dataset.to_csv(DATASET_OUT, index=False)

root - INFO - Loading data
root - INFO - Loading data
root - INFO - Loading data
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Control
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Control
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Control
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide
root - INFO - Loading files from /Users/Praktikum/Documents/HL428/Roiss_L-428_aggregated/Etoposide
root - INFO - Finished loading data.
root - INFO - Finished loading data.
root - INFO - Finished loading data.


# Do quality control
Asses the spectra based on their quality, and remove low quality spectra

In [12]:
path_in = Path(DATASET_OUT)
path_out = Path(RESULT_DIR)

if not os.path.exists(path_out):
    os.makedirs(path_out)

path_out_data = path_out / (path_in.stem + "_qc.csv")
path_out_scores = path_out / (path_in.stem + "_qc_scores.csv")

data = pd.read_csv(path_in)

data_out, _, score_dict = qc.score_sort_spectra(data,
                                                n=QC_NUM,
                                                limits=[QC_LIM_LOW, QC_LIM_HIGH],
                                                bl_method="asls",
                                                sg_window=QC_WINDOW,
                                                threshold=QC_THRESHOLD,
                                                min_height=QC_MIN_HEIGHT,
                                                score_measure=QC_SCORE,
                                                n_peaks_influence=QC_PEAKS,
                                                detailed=True)

visualize = False
if visualize:
    data_vis = data.drop(columns=["label", "file", "group"]).values.astype(float)
    wns_vis = data.drop(columns=["label", "file", "group"]).columns.astype(float)

    rl = RangeLimiter(lim=[QC_LIM_LOW, QC_LIM_HIGH],
                      reference=wns_vis)

    data_rl = rl.fit_transform(data_vis)
    wns_rl = wns_vis[rl.lim_[0]:rl.lim_[1]]

    plot_spectra_peaks(wns_rl,
                       data_rl,
                       score_dict["peak_pos"],
                       labels=score_dict["total_scores"])

data_out.to_csv(path_out_data, index=False)

pd.DataFrame({score_names[QC_SCORE]: score_dict["intensity_scores"],
              "N Peaks": score_dict["peak_scores"]}).to_csv(
    path_out_scores, index=False
)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Preprocess the data

In [13]:
path_in = Path(QC_OUT)
path_out = Path(RESULT_DIR)

filename = path_in.stem.removesuffix("_qc")

if not os.path.exists(path_out):
    os.makedirs(path_out)

path_out = path_out / (filename + "_preprocessed.csv")

data = load_data(QC_OUT)

# save groups and remove column, otherwise preprocess won't work
groups = np.asarray(data.group)
data = data.drop(columns=["group"])

data_prep = preprocess(data, limits=[PREP_LIM_LOW, PREP_LIM_HIGH], sg_window=PREP_WINDOW)

#add groups again
data_prep.insert(2, "group", groups)

data_prep.to_csv(path_out, index=False)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# Implement GroupKFold CV
## LDA Dimensionality Reduction 

In [14]:
path_in = PREP_OUT
#path_out = Path(args.out)

#filename = path_in.stem

data = load_data(path_in)

X = data.drop(columns=["label", "file", "group"])
wns = np.asarray(X.columns.astype(float))
X = np.asarray(X)
y = np.array(data.label)
y, y_key = pd.factorize(y)

#split the dataset according to the groups
groups = np.array(data.group)
logo = LeaveOneGroupOut()



### Baseline with LDA alone

In [15]:
clf = LinearDiscriminantAnalysis()
result = cross_validate(clf, X, y, cv=logo, scoring=SCORING, return_train_score=True, groups=groups)
to_csv(result, path=LDA_DIR + "/lda", scoring=SCORING, param_opt=False)
result

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


{'fit_time': array([0.18045783, 0.27510118, 0.28611994]),
 'score_time': array([0.00653815, 0.00325179, 0.00310922]),
 'test_roc_auc': array([0.88970588, 0.89917695, 0.89797488]),
 'train_roc_auc': array([1., 1., 1.]),
 'test_accuracy': array([0.84166667, 0.86338798, 0.78531073]),
 'train_accuracy': array([1., 1., 1.]),
 'test_f1': array([0.81904762, 0.87684729, 0.7654321 ]),
 'train_f1': array([1., 1., 1.])}

### Feature Selection with PCA followed by LDA

In [16]:
param_grid = {"pca__n_components": range(
    1, 51, 10
)}
clf = Pipeline([("pca", PCA()),
                ("lda", LinearDiscriminantAnalysis())])
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy',
                       return_train_score=True, verbose=3).fit(X, y, groups=groups)

to_csv(grid_rf.cv_results_, scoring=SCORING, path=LDA_DIR + "/pca_lda", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)


Fitting 3 folds for each of 5 candidates, totalling 15 fits
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel

### Non-Negative Matrix Factorization

In [17]:
"""clf = Pipeline([("nmf", NMF(init="nndsvda", tol=1e-2, max_iter=5000)),
                ("lda", LinearDiscriminantAnalysis())])
cross_validate(clf, X, y, cv=logo, groups=groups, scoring=SCORING)"""

'clf = Pipeline([("nmf", NMF(init="nndsvda", tol=1e-2, max_iter=5000)),\n                ("lda", LinearDiscriminantAnalysis())])\ncross_validate(clf, X, y, cv=logo, groups=groups, scoring=SCORING)'

### Feature Agglomeration

In [18]:
param_grid = {"agglo__n_clusters": range(
    5, 41, 5
)}
clf = Pipeline([("agglo", FeatureAgglomeration(connectivity=np.diag(np.ones(len(wns))) +
                                                            np.diag(np.ones(len(wns) - 1), 1) +
                                                            np.diag(np.ones(len(wns) - 1), -1))),
                ("lda", LinearDiscriminantAnalysis())])
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy',
                       return_train_score=True, verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=LDA_DIR + "/fa_lda", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV 1/3] END agglo__n_clusters=5; accuracy: (train=0.828, test=0.875) f1: (train=0.838, test=0.858) roc_auc: (train=0.902, test=0.925) total time=   0.2s
[CV 2/3] END agglo__n_clusters=5; accuracy: (train=0.835, test=0.836) f1: (train=0.816, test=0.833) roc_auc: (train=0.915, test=0.934) total time=   0.2s
[CV 3/3] END agglo__n_clusters=5; accuracy: (train=0.884, test=0.763) f1: (train=0.877, test=0.779) roc_auc: (train=0.931, test=0.849) total time=   0.2s
[CV 1/3] END agglo__n_clusters=10; accuracy: (train=0.886, test=0.929) f1: (train=0.887, test=0.915) roc_auc: (train=0.947, test=0.953) total time=   0.2s
[CV 2/3] END agglo__n_clusters=10; accuracy: (train=0.909, test=0.842) f1: (train=0.897, test=0.836) roc_auc: (train=0.961, test=0.952) total time=   0.2s
[CV 3/3] END agglo__n_clusters=10; accuracy: (train=0.901, test=0.859) f1: (train=0.889, test=0.854) roc_auc: (train=0.959, test=0.915) total time=   0.2s
[CV 1/3] END 

### PeakPicker

In [19]:
param_grid = {"peaks__min_dist": range(
    10, 151, 10
)}
clf = Pipeline([("peaks", PeakPicker()),
                ("lda", LinearDiscriminantAnalysis())])
grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=LDA_DIR + "/peak_lda", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)


Fitting 3 folds for each of 15 candidates, totalling 45 fits
[CV 1/3] END peaks__min_dist=10; accuracy: (train=0.950, test=0.942) f1: (train=0.953, test=0.934) roc_auc: (train=0.990, test=0.980) total time=   0.0s
[CV 2/3] END peaks__min_dist=10; accuracy: (train=0.969, test=0.874) f1: (train=0.966, test=0.880) roc_auc: (train=0.996, test=0.937) total time=   0.0s
[CV 3/3] END peaks__min_dist=10; accuracy: (train=0.955, test=0.881) f1: (train=0.952, test=0.880) roc_auc: (train=0.992, test=0.972) total time=   0.0s
[CV 1/3] END peaks__min_dist=20; accuracy: (train=0.950, test=0.942) f1: (train=0.953, test=0.934) roc_auc: (train=0.989, test=0.980) total time=   0.0s
[CV 2/3] END peaks__min_dist=20; accuracy: (train=0.969, test=0.874) f1: (train=0.966, test=0.880) roc_auc: (train=0.996, test=0.937) total time=   0.0s
[CV 3/3] END peaks__min_dist=20; accuracy: (train=0.950, test=0.893) f1: (train=0.947, test=0.893) roc_auc: (train=0.992, test=0.976) total time=   0.0s
[CV 1/3] END peaks__m

## Regularized Models
### Logistic Regression L1 Penalty

In [20]:
param_grid = {
    "logreg__C": np.logspace(-2, 1, 16)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(solver="liblinear",
                                  penalty="l1", max_iter=1000, random_state=41))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/logreg_l1", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV 1/3] END logreg__C=0.01; accuracy: (train=0.836, test=0.817) f1: (train=0.831, test=0.761) roc_auc: (train=0.884, test=0.882) total time=   0.0s
[CV 2/3] END logreg__C=0.01; accuracy: (train=0.906, test=0.858) f1: (train=0.891, test=0.863) roc_auc: (train=0.979, test=0.907) total time=   0.0s
[CV 3/3] END logreg__C=0.01; accuracy: (train=0.915, test=0.836) f1: (train=0.906, test=0.826) roc_auc: (train=0.977, test=0.909) total time=   0.0s
[CV 1/3] END logreg__C=0.015848931924611134; accuracy: (train=0.883, test=0.929) f1: (train=0.881, test=0.911) roc_auc: (train=0.979, test=0.973) total time=   0.0s
[CV 2/3] END logreg__C=0.015848931924611134; accuracy: (train=0.952, test=0.858) f1: (train=0.947, test=0.879) roc_auc: (train=0.993, test=0.929) total time=   0.0s
[CV 3/3] END logreg__C=0.015848931924611134; accuracy: (train=0.922, test=0.836) f1: (train=0.914, test=0.828) roc_auc: (train=0.983, test=0.928) total time=   0.

### Logistic Regression L2 Penalty

In [21]:
param_grid = {
    "logreg__C": np.logspace(-5, 1, 13)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(solver="liblinear",
                                  penalty="l2", max_iter=1000, random_state=51))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/logreg_l2", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 13 candidates, totalling 39 fits
[CV 1/3] END logreg__C=1e-05; accuracy: (train=0.883, test=0.921) f1: (train=0.883, test=0.905) roc_auc: (train=0.981, test=0.979) total time=   0.0s
[CV 2/3] END logreg__C=1e-05; accuracy: (train=0.918, test=0.896) f1: (train=0.909, test=0.899) roc_auc: (train=0.984, test=0.966) total time=   0.0s
[CV 3/3] END logreg__C=1e-05; accuracy: (train=0.917, test=0.853) f1: (train=0.909, test=0.845) roc_auc: (train=0.981, test=0.961) total time=   0.0s
[CV 1/3] END logreg__C=3.1622776601683795e-05; accuracy: (train=0.892, test=0.938) f1: (train=0.891, test=0.925) roc_auc: (train=0.985, test=0.980) total time=   0.0s
[CV 2/3] END logreg__C=3.1622776601683795e-05; accuracy: (train=0.928, test=0.896) f1: (train=0.920, test=0.901) roc_auc: (train=0.988, test=0.969) total time=   0.0s
[CV 3/3] END logreg__C=3.1622776601683795e-05; accuracy: (train=0.929, test=0.853) f1: (train=0.923, test=0.845) roc_auc: (train=0.984, test=0.969) total t

### Linear SVM L1 Penalty

In [22]:
param_grid = {
    "svm__C": np.logspace(-3, 0, 16)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", LinearSVC(penalty="l1", dual=False, max_iter=10000))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/svm_l1", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV 1/3] END svm__C=0.001; accuracy: (train=0.456, test=0.567) f1: (train=0.000, test=0.000) roc_auc: (train=0.500, test=0.500) total time=   0.0s
[CV 2/3] END svm__C=0.001; accuracy: (train=0.525, test=0.443) f1: (train=0.000, test=0.000) roc_auc: (train=0.500, test=0.500) total time=   0.0s
[CV 3/3] END svm__C=0.001; accuracy: (train=0.513, test=0.469) f1: (train=0.000, test=0.000) roc_auc: (train=0.500, test=0.500) total time=   0.0s
[CV 1/3] END svm__C=0.001584893192461114; accuracy: (train=0.456, test=0.567) f1: (train=0.000, test=0.000) roc_auc: (train=0.500, test=0.500) total time=   0.0s
[CV 2/3] END svm__C=0.001584893192461114; accuracy: (train=0.525, test=0.443) f1: (train=0.000, test=0.000) roc_auc: (train=0.500, test=0.500) total time=   0.0s
[CV 3/3] END svm__C=0.001584893192461114; accuracy: (train=0.513, test=0.469) f1: (train=0.000, test=0.000) roc_auc: (train=0.500, test=0.500) total time=   0.0s
[CV 1/3] END

### Linear SVM L2 Penalty

In [23]:
param_grid = {
    "svm__C": np.logspace(-5, -1, 13)
}
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", LinearSVC(penalty="l2", max_iter=5000))
])

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=REG_DIR + "/svm_l2", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 13 candidates, totalling 39 fits
[CV 1/3] END svm__C=1e-05; accuracy: (train=0.908, test=0.946) f1: (train=0.910, test=0.935) roc_auc: (train=0.989, test=0.981) total time=   0.0s
[CV 2/3] END svm__C=1e-05; accuracy: (train=0.945, test=0.902) f1: (train=0.940, test=0.906) roc_auc: (train=0.993, test=0.970) total time=   0.0s
[CV 3/3] END svm__C=1e-05; accuracy: (train=0.936, test=0.864) f1: (train=0.931, test=0.857) roc_auc: (train=0.988, test=0.976) total time=   0.1s
[CV 1/3] END svm__C=2.1544346900318823e-05; accuracy: (train=0.917, test=0.950) f1: (train=0.919, test=0.941) roc_auc: (train=0.993, test=0.981) total time=   0.0s
[CV 2/3] END svm__C=2.1544346900318823e-05; accuracy: (train=0.954, test=0.907) f1: (train=0.950, test=0.912) roc_auc: (train=0.995, test=0.969) total time=   0.0s
[CV 3/3] END svm__C=2.1544346900318823e-05; accuracy: (train=0.943, test=0.870) f1: (train=0.939, test=0.861) roc_auc: (train=0.992, test=0.982) total time=   0.0s
[CV 1/

## Tree-based models
### Basic Decision Tree


In [24]:
param_grid = {
    "ccp_alpha": np.logspace(-3, -1, 9)
}
clf = DecisionTreeClassifier(random_state=653)

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=TREE_DIR + "/decision_tree", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV 1/3] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.817) f1: (train=1.000, test=0.807) roc_auc: (train=1.000, test=0.825) total time=   0.5s
[CV 2/3] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.770) f1: (train=1.000, test=0.816) roc_auc: (train=1.000, test=0.752) total time=   0.4s
[CV 3/3] END ccp_alpha=0.001; accuracy: (train=1.000, test=0.847) f1: (train=1.000, test=0.844) roc_auc: (train=1.000, test=0.852) total time=   0.4s
[CV 1/3] END ccp_alpha=0.0017782794100389228; accuracy: (train=1.000, test=0.817) f1: (train=1.000, test=0.807) roc_auc: (train=1.000, test=0.825) total time=   0.4s
[CV 2/3] END ccp_alpha=0.0017782794100389228; accuracy: (train=1.000, test=0.770) f1: (train=1.000, test=0.816) roc_auc: (train=1.000, test=0.752) total time=   0.4s
[CV 3/3] END ccp_alpha=0.0017782794100389228; accuracy: (train=1.000, test=0.847) f1: (train=1.000, test=0.844) roc_auc: (train=1.000, test=0.852) total time=

### Random Forest

In [25]:
param_grid = {
    "colsample_bytree": np.linspace(0.01, 0.2, 20)
}
clf = LGBMClassifier(boosting_type="rf",
                     subsample=0.8,
                     subsample_freq=1,
                     max_bin=10,
                     max_depth=8,
                     random_state=2434)

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=TREE_DIR + "/random_forest", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV 1/3] END colsample_bytree=0.01; accuracy: (train=0.975, test=0.971) f1: (train=0.977, test=0.967) roc_auc: (train=0.998, test=0.991) total time=   0.3s
[CV 2/3] END colsample_bytree=0.01; accuracy: (train=0.988, test=0.934) f1: (train=0.987, test=0.942) roc_auc: (train=1.000, test=0.979) total time=   0.3s
[CV 3/3] END colsample_bytree=0.01; accuracy: (train=0.976, test=0.887) f1: (train=0.975, test=0.885) roc_auc: (train=0.999, test=0.965) total time=   0.4s
[CV 1/3] END colsample_bytree=0.02; accuracy: (train=0.978, test=0.958) f1: (train=0.979, test=0.952) roc_auc: (train=0.998, test=0.987) total time=   0.3s
[CV 2/3] END colsample_bytree=0.02; accuracy: (train=0.988, test=0.885) f1: (train=0.987, test=0.903) roc_auc: (train=1.000, test=0.973) total time=   0.7s
[CV 3/3] END colsample_bytree=0.02; accuracy: (train=0.979, test=0.881) f1: (train=0.978, test=0.879) roc_auc: (train=1.000, test=0.969) total time=   0.6s
[CV

### Gradient-boosted Decision Tree

In [26]:
param_grid = {
    "learning_rate": np.linspace(0.01, 0.2, 20)
}
clf = LGBMClassifier(colsample_bytree=0.2,
                     max_bin=10,
                     max_depth=5,
                     random_state=6233)

grid_rf = GridSearchCV(clf, param_grid=param_grid, cv=logo, scoring=SCORING, refit='accuracy', return_train_score=True,
                       verbose=3).fit(X, y, groups=groups)
to_csv(grid_rf.cv_results_, scoring=SCORING, path=TREE_DIR + "/gbdt", param_opt=True)

print(" Results from Grid Search ")
print("\n The best estimator across ALL searched params:\n", grid_rf.best_estimator_)
print("\n The best score across ALL searched params:\n", grid_rf.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_rf.best_params_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV 1/3] END learning_rate=0.01; accuracy: (train=0.969, test=0.871) f1: (train=0.972, test=0.861) roc_auc: (train=0.998, test=0.970) total time=   0.7s
[CV 2/3] END learning_rate=0.01; accuracy: (train=0.981, test=0.743) f1: (train=0.980, test=0.802) roc_auc: (train=0.999, test=0.909) total time=   0.8s
[CV 3/3] END learning_rate=0.01; accuracy: (train=0.986, test=0.864) f1: (train=0.986, test=0.870) roc_auc: (train=1.000, test=0.961) total time=   0.5s
[CV 1/3] END learning_rate=0.02; accuracy: (train=0.994, test=0.946) f1: (train=0.995, test=0.939) roc_auc: (train=1.000, test=0.985) total time=   0.8s
[CV 2/3] END learning_rate=0.02; accuracy: (train=0.993, test=0.765) f1: (train=0.992, test=0.820) roc_auc: (train=1.000, test=0.915) total time=   0.8s
[CV 3/3] END learning_rate=0.02; accuracy: (train=1.000, test=0.904) f1: (train=1.000, test=0.905) roc_auc: (train=1.000, test=0.970) total time=   0.8s
[CV 1/3] END learning